# DQMC


## Hamiltonian & Partition function


Use 2d square-lattice Hubbard model as an example here. For detailed analytical derivation, please check https://quantummc.xyz/2020/10/11/dqmc-note/ .


The half-filling Hamiltonian is: $$H=-t\mathop{\sum}\limits_{<i,j>\sigma}c_{i\sigma}^\dagger c_{j\sigma}+h.c.+U\mathop{\sum}\limits_i (n_{i\uparrow}-\frac{1}{2})(n_{i\downarrow}-\frac{1}{2}).$$

After Trotter decomposition, the partition is $$Tr\{e^{-\beta H}\}=Tr\{(e^{-\Delta\tau H})^{L_\tau}\}$$

For the kinetic energy, the sum for spin-up and spin-down are the same and for example it can be expressed as :
$$H_0=c^\dagger_\uparrow T c_\uparrow=-t\mathop{\sum}\limits_{<i,j>}c_{i\uparrow}^\dagger c_{j\uparrow}+h.c. $$

For the interaction,  it can be expressed as :
$$H_U=U\mathop{\sum}\limits_i (n_{i\uparrow}-\frac{1}{2})(n_{i\downarrow}-\frac{1}{2})=-\frac{U}{2}(n_\uparrow-n_\downarrow)^2+\frac{U}{4}$$
After HS transformation: $$e^{-\Delta\tau H_U}=\gamma\sum_{s=\pm 1}e^{\alpha s(n_\uparrow-n_\downarrow)},$$ with $\gamma=\frac{1}{2}e^{-\Delta\tau U/4}$, and $\cosh(\alpha)=e^{\Delta\tau U/2}$.

Therefore, the expanded partition function is:
$$Z=$$

